This code loads the dataset and plot rectangles according to the curated XML

In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
import xml.etree.ElementTree as ET
from PIL import Image, ImageOps
import numpy as np
import os.path
import os
%matplotlib inline

In [2]:
def crop_data(folder_in = 'raw data/' ,folder_out = 'croped data/', print_boxes = False, rotations_img = 0, mirror_img = True):

    malaria_images_counter = 0
    counter_nomalaria_img = 0
    for file in os.listdir('./'+folder_in):

        if file.endswith('.jpg'):
            name = file[0:-4:]
            #print(name)
            if os.path.isfile(folder_in+name+'.xml'):
                
                #print(name+'.xml') #(deletable)
                # check the xml file to find the boxes
                tree = ET.parse(folder_in+name+'.xml')
                boxes = tree.findall('./object/bndbox')

                #load each image
                image = Image.open(folder_in+name+'.jpg')
                counter = 0  # that's a counter of how mnany dots there are in each image, you might not need it
                diagnosed_points = {}

                #this part crops the malaria dots
                for box in boxes:
                    counter += 1

                    #get the corner of the box from the xml
                    xmin = float(box.find('xmin').text)
                    ymin = float(box.find('ymin').text)
                    xmax = float(box.find('xmax').text)
                    ymax = float(box.find('ymax').text) 

                    file_name = str(name)+' '+str(round(xmin,2))+' '+str(round(ymin,2))

                    # this is for croping the parts without malaria
                    diagnosed_points[counter] = {'xmin':xmin,'ymin':ymin,'xmax':xmax,'ymax':ymax} 
                    if ((xmin) <= image.size[0] and xmax<= image.size[0] and (ymin) <= image.size[1] and (ymax) <= image.size[1] and (xmax-xmin)==(ymax-ymin)):
                        # crop where malaria is
                        if (ymin < image.size[1] and xmax < image.size[0] and xmin >= 0 and ymin >=0 ):  #check if the images are square
                            img = image.crop((xmin,ymin, xmax,ymax))
                            #print(xmin, xmax,ymin,ymax)
                            # rotate mirror and save images 
                            for i in range(0,rotations_img+1): 
                                temp_img = img.rotate(i*90)
                                temp_img.save(folder_out+'malaria/'+str(file_name)+' malaria '+str(i)+'deg'+'.jpg')
                                if mirror_img == True:
                                    mirrored_img = ImageOps.mirror(temp_img)
                                    mirrored_img.save(folder_out+'malaria/'+str(file_name)+' malaria '+str(i)+'deg mirror'+'.jpg')
                                malaria_images_counter += 1
                                
                #this part crops the parts without malaria
                if print_boxes == True:
                    #this part checks grafically if the part to rhop touches a part with malaria
                    #load each image
                    imag = np.array(Image.open(folder_in+name+'.jpg'), dtype=np.uint8)
                    # Create figure and axes
                    fig,ax = plt.subplots(1)
                    # Display the image
                    ax.imshow(imag)
                    
                # travel all over the image
                for x in range(0,image.size[0],40):
                    for y in range (20,image.size[1],30):
                        # check if what we want to crop is completely inside the image
                        if ((x + 40)<=image.size[0] and (y + 40)<=image.size[1]):
                            checker = True
                            #checks if the square to crop colides with a square with malaria
                            for i in range(1,len(diagnosed_points)+1):

                                if ((x >= diagnosed_points[i]['xmin'] and x <= diagnosed_points[i]['xmax']) and 
                                     (y >= diagnosed_points[i]['ymin'] and y <= diagnosed_points[i]['ymax']) or

                                    ((x +40)>= diagnosed_points[i]['xmin'] and (x +40) <= diagnosed_points[i]['xmax']) and 
                                     (y >= diagnosed_points[i]['ymin'] and y <= diagnosed_points[i]['ymax']) or 

                                    (x >= diagnosed_points[i]['xmin'] and x <= diagnosed_points[i]['xmax']) and 
                                     ((y + 40) >= diagnosed_points[i]['ymin'] and (y + 40) <= diagnosed_points[i]['ymax']) or 

                                    ((x + 40) >= diagnosed_points[i]['xmin'] and (x +40) <= diagnosed_points[i]['xmax']) and 
                                     ((y + 40) >= diagnosed_points[i]['ymin'] and (y + 40) <= diagnosed_points[i]['ymax'])
                                   ):
                                    checker = False
                                    if print_boxes == True:
                                        #this part checks grafically if the part to crop touches a part with malaria
                                        rect = patches.Rectangle((x,y),40,40,linewidth=1,edgecolor='g',facecolor='none')
                                        # Add the patch to the Axes
                                        ax.add_patch(rect)
                                        
                            # if the square to crop doesnt colide with a square with malaria then crop it
                            if (checker is True):

                                # crop image
                                img = image.crop((x,y, x +40,y+40))
                                file_name = str(name)+' '+str(x)+' '+str(y)
                                mean_image = mpimg.pil_to_array(img.convert('LA'))[:,:,0]
                                bias = max(mean_image.reshape(40*40))-min(mean_image.reshape(40*40))
                                mean_image = mean_image.reshape(1,mean_image.shape[0]*mean_image.shape[1]).mean(axis=1)
                                
            
                            # rotate, mirror and save images
                                #antes era  <= 130
                                if mean_image <= 130 and  counter_nomalaria_img <= malaria_images_counter: # the smaller the mean the darker the images
                                    for i in range(0,rotations_img+1): # (0,271,90)
                                        #print(bias)
                                        counter_nomalaria_img += 1
                                        temp_img = img.rotate(i*90)
                                        temp_img.save(folder_out+'no malaria/'+str(file_name)+' no malaria '+str(i)+'deg'+'.jpg')
                                        if mirror_img == True:
                                            mirrored_img = ImageOps.mirror(temp_img)
                                            mirrored_img.save(folder_out+'no malaria/'+str(file_name)+' no malaria '+str(i)+'deg mirror'+'.jpg')            
                if print_boxes == True:
                    #this part checks grafically if the part to crop touches a part with malaria
                    plt.xticks(())
                    plt.yticks(())
                    plt.show() 
    print('Result')
    print('total malaria croped images: ',malaria_images_counter)
    print('total no malaria croped images: ',counter_nomalaria_img)



In [3]:
crop_data(folder_in = 'raw data/' ,folder_out = 'croped data/', print_boxes = False, 
          rotations_img = 0, mirror_img = False)

Result
total malaria croped images:  5497
total no malaria croped images:  5498
